In [ ]:
import torch
import torchaudio
from IPython.display import Audio, display
from speechbrain.lobes.augment import TimeDomainSpecAugment

from s3prl.downstream.augment_utils import TempoPerturbation, AddNoise, VLTP
from s3prl.downstream.mdd.dataset import L2ArcticDataset

In [ ]:
dataset = L2ArcticDataset(
    'train',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/'
)

In [ ]:
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  display(Audio(waveform, rate=sample_rate))

In [ ]:
audio = torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000)

play_audio(audio, 16000)

In [ ]:
add_noise = AddNoise(
    prob=0.9,
    snr_low=0,
    snr_high=1,
    device="cpu",
)


timedomain_specaug = TimeDomainSpecAugment(
    speeds=[95, 100, 105],
    sample_rate=16000,
)

tempo_perturbation = TempoPerturbation(
    N=1024,
    sample_rate=16000,
    alpha_high=1.2,
    alpha_low=0.5,
    delta_max=512,
    prob=1.0,
)

In [ ]:
torch.manual_seed(1337)
audio = torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000)

audio = tempo_perturbation([audio])[0]
# audio = vltp([audio])[0]

audio = timedomain_specaug(
    torch.nn.utils.rnn.pad_sequence([audio], batch_first=True),
    torch.tensor([len(audio)], dtype=torch.long))

print(audio.shape)

lens = torch.tensor([len(audio[0])], dtype=torch.long)
audio = add_noise(
        audio,
        lens,
    )[0][:lens[0]]


In [ ]:
play_audio(
    audio,
    sample_rate=16000,
)